## Package Imports

In [44]:
import base64
import json
import os
import platform
import requests
import tarfile
import zipfile
import uuid
import pandas as pd

from IPython.core.display import display, HTML

## Setup CPD CTL

In [45]:
#DOWNLOAD CPDCTL

PLATFORM = platform.system().lower()
CPDCTL_ARCH = "{}_amd64".format(PLATFORM)
CPDCTL_RELEASES_URL="https://api.github.com/repos/IBM/cpdctl/releases"
CWD = os.getcwd()
PATH = os.environ['PATH']
CPDCONFIG = os.path.join(CWD, '.cpdctl.config.yml')

response = requests.get(CPDCTL_RELEASES_URL)
assets = response.json()[0]['assets']
platform_asset = next(a for a in assets if CPDCTL_ARCH in a['name'])
cpdctl_url = platform_asset['url']
cpdctl_file_name = platform_asset['name']

response = requests.get(cpdctl_url, headers={'Accept': 'application/octet-stream'})
with open(cpdctl_file_name, 'wb') as f:
    f.write(response.content)
    
display(HTML('<code>cpdctl</code> binary downloaded from: <a href="{}">{}</a>'.format(platform_asset['browser_download_url'], platform_asset['name'])))

In [46]:
%%capture

%env PATH={CWD}:{PATH}
%env CPDCONFIG={CPDCONFIG}

In [47]:
if cpdctl_file_name.endswith('tar.gz'):
    with tarfile.open(cpdctl_file_name, "r:gz") as tar:
        tar.extractall()
elif cpdctl_file_name.endswith('zip'):
    with zipfile.ZipFile(cpdctl_file_name, 'r') as zf:
        zf.extractall()

if CPDCONFIG and os.path.exists(CPDCONFIG):
    os.remove(CPDCONFIG)
    
version_r = ! cpdctl version
CPDCTL_VERSION = version_r.s

print("cpdctl version: {}".format(CPDCTL_VERSION))

cpdctl version: 1.0.79


In [48]:
CPD_URL = 'https://zen-cpd-zen.cpd-msk-dal10-2bef1f4b4097001da9502000c44fc2b2-0000.us-south.containers.appdomain.cloud'

In [49]:
! cpdctl config profile set cpd --url {CPD_URL}

In [8]:
#! cpdctl config user set cpd_admin --username {CPD_USER_NAME} --password {CPD_USER_PASSWORD}
#! cpdctl config context set cpd --profile cpd --user cpd_user
#! cpdctl config context list
#! cpdctl config context use cpd
#! cpdctl project list --output json

## Get Project Source

In [50]:
PROJECT_ZIP_FILE_NAME = 'AutoAIIncidentResolution.zip'

In [51]:
!rm AutoAIIncidentResolution.zip
!wget https://github.com/ricdodds/ml101/raw/main/projects/AutoAIIncidentResolution.zip
! unzip -l AutoAIIncidentResolution.zip

--2021-06-17 17:06:38--  https://github.com/ricdodds/ml101/raw/main/projects/AutoAIIncidentResolution.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ricdodds/ml101/main/projects/AutoAIIncidentResolution.zip [following]
--2021-06-17 17:06:39--  https://raw.githubusercontent.com/ricdodds/ml101/main/projects/AutoAIIncidentResolution.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4595994 (4.4M) [application/zip]
Saving to: ‘AutoAIIncidentResolution.zip’

AutoAIIncidentResol 100%[===================>]   4.38M  --.-KB/s    in 0.06s   

2021-06-17 17:06:39 (79.6 MB/s) - ‘AutoAIIncidentResolut

## Create Project for Users

In [52]:
PROJECT_NAME = 'AutoAIIncidentResolution'

In [53]:
# GET PROJECTS AND DELETE ANY WITH SAME IMPORT NAME. MAY SWITCH TO CREATE PROJECT WITH NAME + SEQUENCE NUMBER
def delete_project(project_name):
    JMES_QUERY = "resources[?entity.name == '{}'].metadata.guid".format(project_name)
    result = ! cpdctl project list --output json --jmes-query "{JMES_QUERY}"
    project_ids = json.loads(result.s)
    if project_ids:
        for project_id in project_ids:
            print('Deleting project with ID: {}'.format(project_id))
            ! cpdctl project delete --project-id {project_id}

In [54]:
def create_project(project_name):
    STORAGE = {"type": "assetfiles", "guid": str(uuid.uuid4())}
    STORAGE_JSON = json.dumps(STORAGE)
    result = ! cpdctl project create --name {project_name} --output json --raw-output --storage '{STORAGE_JSON}' --jmes-query 'location'
    res_project_id = result.s.split('/')[-1]
    print("Created new '{}' project. ID is: {}".format(project_name, res_project_id))
    return res_project_id

In [55]:
def import_zip_assets(project_id, project_file_name):
    result = ! cpdctl asset import start --project-id {project_id} --import-file {project_file_name} --output json --jmes-query "metadata.id" --raw-output
    import_id = result.s
    print("Import project zip. ID is: {}".format(import_id))
    return import_id

In [56]:
def check_asset_import_state(project_id, import_id):
    ! cpdctl asset import get --project-id {project_id} --import-id {import_id}


In [57]:
def list_project_assets(project_id):
    ! cpdctl asset search --query '*:*' --type-name asset --project-id {project_id}


In [58]:
def validate_project_created(project_name):
    res = ! cpdctl project list --output json
    projects = json.loads(res.s)
    project_found = False
    for project in projects['resources']:
        if project_name == project['entity']['name']:
            project_found = True
            break
    return project_found

In [59]:
df = pd.read_csv('/project_data/data_asset/passwords.csv')
#d = {'username': ['demouser1', 'demouser2'], 'password': ['demouser1', 'demouser2'], 'email': ['jrtorres@us.ibm.com', 'jrtorres@us.ibm.com']}
#df = pd.DataFrame(data=d)
df.head(10)

username           password              email
0  user000    81-none-hole-41  user000@email.com
1  user001  26-count-level-05  user001@email.com
2  user002    07-loss-arms-35  user002@email.com
3  user003   91-brown-king-51  user003@email.com
4  user004  06-shine-clear-60  user004@email.com
5  user005  03-large-horse-92  user005@email.com
6  user006   26-wore-track-17  user006@email.com
7  user007    77-jump-half-46  user007@email.com
8  user008    76-left-shoe-67  user008@email.com
9  user009    99-grew-says-67  user009@email.com

In [61]:
for index, row in df.iterrows():
    user = 'cpd_user_' + str(index)
    context = 'cpd_cont_user_' + str(index)
    ! cpdctl config user set {user} --username {row['username']} --password {row['password']}
    ! cpdctl config context set {context} --profile cpd --user {user}
    ! cpdctl config context use {context}
    delete_project(PROJECT_NAME)
    proj_id = create_project(PROJECT_NAME)
    imp_id = import_zip_assets(proj_id, PROJECT_ZIP_FILE_NAME)
    check_asset_import_state(proj_id, imp_id)
    proj_created = validate_project_created(PROJECT_NAME)
    list_project_assets(proj_id)

Switched to context "cpd_cont_user_0".
Created new 'AutoAIIncidentResolution' project. ID is: 7fedbb8e-2c17-411c-9b1e-713c518192a8
Import project zip. ID is: e2014c73-25d0-4b06-9ad4-9f6da1bc8a3f
...
              
ID:        e2014c73-25d0-4b06-9ad4-9f6da1bc8a3f   
Created:   2021-06-17T17:08:07.639Z   
State:     completed   
...
ID                                     Name                                                 Created                    Description   Type           State       Tags   Size   
6d39ef68-8ad1-4733-813a-0d3eba9d4fd3   incident_sla.csv                                     2021-06-17T17:08:15.000Z                 data_asset     available   []     21054333   
6c3e1f5d-aebd-4bbf-be4b-c43a91b09171   IncidentResolutionAutoAI                             2021-06-17T17:08:15.000Z                 wml_pipeline   available   []     0   
1ffd6293-85a6-4a39-ba5f-1afb0c84eb5e   IncidentResolutionAutoAI - P4 XGBRegressorEstimat…   2021-06-17T17:08:16.000Z                 wml_model

Switched to context "cpd_cont_user_6".
Deleting project with ID: 30a3e92b-87ca-4dc6-930d-acba424cf23e
...
OK
Created new 'AutoAIIncidentResolution' project. ID is: 5f4cc9a0-285d-42cb-b704-64a013f3970a
Import project zip. ID is: 1d687cfe-7e36-4575-9c42-f4fe3528c401
...
              
ID:        1d687cfe-7e36-4575-9c42-f4fe3528c401   
Created:   2021-06-17T17:11:23.736Z   
State:     completed   
...
ID                                     Name                                                 Created                    Description   Type           State       Tags   Size   
f617650c-a3de-4bfb-b399-7f7fb83b2f7f   incident_sla.csv                                     2021-06-17T17:11:31.000Z                 data_asset     available   []     21054333   
4fab4d77-3f5f-47b5-8fb2-3049ab9e9b72   incident.csv                                         2021-06-17T17:11:31.000Z                 data_asset     available   []     9750386   
f3f47910-98fd-4c86-9060-bacbe3ebc146   IncidentResolutionAutoAI   

KeyError: 'resources'

## Setup Deployment Spaces

In [20]:
DEPLOYMENT_SPACE_NAME = 'IncidentResolutionDeploymentSpace'
DEPLOYMENT_SPACE_DESCRIPTION = 'Deployment Space for incident resolution workshop'

In [21]:
# HARDCODING UNTIL FIGURE OUT HOW TO GET ADMIN ID TO BUILD MEMBER
ADMIN_USER_ID = '1000330999'
MEMBER = [{"id": ADMIN_USER_ID, "role": "admin", "state": "active", "type": "user"}]
MEMBER_JSON = json.dumps(MEMBER)

In [23]:
# GET SPACES AND DELETE ANY WITH SAME NAME WE ARE USING. MAY NOT WANT TO DO, BUT SPACE CREATION WILL FAIL IF NAME ALREADY EXISTS.
def delete_space(space_name):
    SPACES_QUERY = "resources[?entity.name == '{}'].metadata.id".format(space_name)
    result = ! cpdctl space list --output json --jmes-query "{SPACES_QUERY}"
    space_ids = json.loads(result.s)
    if space_ids:
        for space_id in space_ids:
            print('Deleting space with ID: {}'.format(space_id))
            ! cpdctl space delete --space-id {space_id}

In [24]:
res = ! cpdctl config context list --output json
contexts = json.loads(res.s)
for context in contexts:
    c_name = context['Name']
    ! cpdctl config context use {c_name}
    ! cpdctl space list
    delete_space(DEPLOYMENT_SPACE_NAME)
    dsc_result = ! cpdctl space create --name {DEPLOYMENT_SPACE_NAME} --description {DEPLOYMENT_SPACE_DESCRIPTION} --output json
    dep_space = json.loads(dsc_result.s)
    deployment_space_id = dep_space['metadata']['id']
    print("Created new '{}' deployment space. ID is: {}".format(DEPLOYMENT_SPACE_NAME, deployment_space_id))
#     ! cpdctl space member create --space-id {deployment_space_id} --members '{MEMBER_JSON}' 
#     ! cpdctl space member list --space-id {deployment_space_id}

Switched to context "cpd_cont_user_0".
...
Nothing to show.
Created new 'IncidentResolutionDeploymentSpace' deployment space. ID is: b5bed407-3f8c-4191-af91-598f54871c26
Switched to context "cpd_cont_user_1".
...
Nothing to show.
Created new 'IncidentResolutionDeploymentSpace' deployment space. ID is: 00a0c7cc-23cc-4c2f-9885-82a2d10319bc
Switched to context "cpd_cont_user_2".
...
Nothing to show.
Created new 'IncidentResolutionDeploymentSpace' deployment space. ID is: b88f78ff-241f-41df-809a-61218a4452cf
Switched to context "cpd_cont_user_3".
...
Nothing to show.
Created new 'IncidentResolutionDeploymentSpace' deployment space. ID is: 1006e320-2b8c-4af2-b5eb-244ef14bce24
Switched to context "cpd_cont_user_4".
...
Nothing to show.
Created new 'IncidentResolutionDeploymentSpace' deployment space. ID is: a362e10e-9555-4c77-a548-50ff471ebd17
Switched to context "cpd_cont_user_5".
...
Nothing to show.
Created new 'IncidentResolutionDeploymentSpace' deployment space. ID is: d5fa9770-9f3d-418

## Extra Cleanup Helpers

In [25]:
# REMOVE ALL CONTEXTS
res = ! cpdctl config context list --output json
contexts = json.loads(res.s)
for context in contexts:
    c_name = context['Name']
    print("Unsetting context: {}".format(c_name))
    ! cpdctl config context unset {c_name}
    
! cpdctl config context list

Unsetting context: cpd_cont_user_0
Unsetting context: cpd_cont_user_1
Unsetting context: cpd_cont_user_2
Unsetting context: cpd_cont_user_3
Unsetting context: cpd_cont_user_4
Unsetting context: cpd_cont_user_5
Unsetting context: cpd_cont_user_6
Unsetting context: cpd_cont_user_7
Unsetting context: cpd_cont_user_8
Unsetting context: cpd_cont_user_9
Unsetting context: cpd_cont_user_10
Unsetting context: cpd_cont_user_11
Unsetting context: cpd_cont_user_12
Unsetting context: cpd_cont_user_13
Unsetting context: cpd_cont_user_14
Unsetting context: cpd_cont_user_15
Unsetting context: cpd_cont_user_16
Unsetting context: cpd_cont_user_17
Unsetting context: cpd_cont_user_18
Unsetting context: cpd_cont_user_19
Unsetting context: cpd_cont_user_20
Unsetting context: cpd_cont_user_21
Unsetting context: cpd_cont_user_22
Unsetting context: cpd_cont_user_23
Unsetting context: cpd_cont_user_24
Unsetting context: cpd_cont_user_25
Unsetting context: cpd_cont_user_26
Unsetting context: cpd_cont_user_27
Un

In [26]:
# REMOVE ALL USERS
res = ! cpdctl config user list --output json
users = json.loads(res.s)
for user in users:
    u_name = user['Name']
    print("Unsetting user: {}".format(u_name))
    ! cpdctl config user unset {u_name}
    
! cpdctl config user list 

Unsetting user: cpd_user_0
Unsetting user: cpd_user_1
Unsetting user: cpd_user_2
Unsetting user: cpd_user_3
Unsetting user: cpd_user_4
Unsetting user: cpd_user_5
Unsetting user: cpd_user_6
Unsetting user: cpd_user_7
Unsetting user: cpd_user_8
Unsetting user: cpd_user_9
Unsetting user: cpd_user_10
Unsetting user: cpd_user_11
Unsetting user: cpd_user_12
Unsetting user: cpd_user_13
Unsetting user: cpd_user_14
Unsetting user: cpd_user_15
Unsetting user: cpd_user_16
Unsetting user: cpd_user_17
Unsetting user: cpd_user_18
Unsetting user: cpd_user_19
Unsetting user: cpd_user_20
Unsetting user: cpd_user_21
Unsetting user: cpd_user_22
Unsetting user: cpd_user_23
Unsetting user: cpd_user_24
Unsetting user: cpd_user_25
Unsetting user: cpd_user_26
Unsetting user: cpd_user_27
Unsetting user: cpd_user_28
Unsetting user: cpd_user_29
Nothing to show.
